In [10]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import torch
import time
import xarray as xr
import pickle
import matplotlib.pyplot as plt
from scipy.linalg import cholesky
import sys
from os.path import dirname, join, abspath

from brainmodel_utils.core.constants import RIDGECV_ALPHA_CV
from brainmodel_utils.neural_mappers import PercentileNeuralMap
from brainmodel_utils.neural_mappers.utils import (
    generate_train_test_splits,
    convert_dict_to_tuple,
)

from brainmodel_utils.metrics.consistency import get_linregress_consistency,get_linregress_consistency_persphalftrial
sys.path.insert(0, abspath(join(dirname("utils.py"), '..')))
from zfa.data_comparisons.utils import load_data_tensors

#from zfa.core.default_dirs import (
#    BASE_DIR,
#    NEURAL_TRIALS_PATH,
#    GLIAL_TRIALS_PATH,
#    INTER_ANIMAL_RESULTS_DIR,
#)

# Ensure PyTorch is using a GPU
#if not torch.cuda.is_available():
    #raise RuntimeError("PyTorch cannot find a GPU!")

#device = torch.device("cuda:0")



In [8]:
DATA_ROOT = "/data/group_data/neuroagents_lab/neural_datasets/zfa_data"
BASE_ROOT = "/data/user_data/rdkeller/"
BASE_DIR = os.path.join(BASE_ROOT, "zfa/")
WHITE_NOISE_TRIALS_PATH = os.path.join(BASE_DIR, "white_noise.pickle")
GP_TRIALS_PATH = os.path.join(BASE_DIR, "gp.pickle")
BANG_BANG_TRIALS_PATH = os.path.join(BASE_DIR, "bang_bang.pickle")

In [12]:
neural_brain_data, glial_brain_data, white_noise_data, gp_data, bb_data = load_data_tensors()
ANIMALS = list(glial_brain_data.keys())
target_animal = ANIMALS[1]
target_brain_data = glial_brain_data[target_animal]

 ---Loading data tensors---  


In [14]:
target_brain_data

<xarray.DataArray (trials: 224, time: 20, units: 5000)> Size: 90MB
array([[[-1.09767914e-03,  1.54304504e-03,  2.21443176e-03, ...,
         -5.11169434e-03, -4.77981567e-03,  9.17053223e-03],
        [-5.38253784e-03, -8.16345215e-04,  2.26616859e-04, ...,
         -2.20489502e-03, -3.19671631e-03,  3.45230103e-03],
        [-5.66101074e-03, -4.57763672e-03, -6.06536865e-03, ...,
          4.82940674e-03, -2.69508362e-03,  8.26358795e-04],
        ...,
        [ 3.78799438e-03,  1.70612335e-03,  1.20468140e-02, ...,
          5.55419922e-02,  5.72509766e-02,  7.00073242e-02],
        [ 1.31759644e-02,  1.80244446e-03,  2.27050781e-02, ...,
          6.56738281e-02,  6.55517578e-02,  7.50122070e-02],
        [ 4.52041626e-03, -2.26616859e-04,  8.06427002e-03, ...,
          4.54711914e-02,  6.29272461e-02,  4.95300293e-02]],

       [[-4.72259521e-03, -6.53839111e-03, -3.93676758e-03, ...,
          6.38198853e-03, -5.32531738e-03,  7.26318359e-03],
        [ 4.45556641e-03, -3.84807587e-04,  9.93347168e-03, ...,
          4.02450562e-03, -4.31442261e-03, -5.83648682e-03],
        [ 4.21905518e-03, -5.67626953e-03,  9.05513763e-04, ...,
          2.28309631e-03, -1.04217529e-02, -1.00910664e-04],
...
          9.46807861e-03, -5.98907471e-04, -7.24792480e-04],
        [-4.78363037e-03, -6.96182251e-03, -3.38172913e-03, ...,
          1.01623535e-02, -6.74247742e-04,  3.77464294e-03],
        [-2.94923782e-04, -2.40898132e-03,  1.34277344e-02, ...,
          8.56018066e-03,  7.11059570e-03,  1.46865845e-03]],

       [[ 2.29454041e-03, -6.67953491e-03,  9.11951065e-05, ...,
          1.46865845e-03,  9.28401947e-04, -2.16293335e-03],
        [-5.67245483e-03, -6.42013550e-03,  4.24575806e-03, ...,
          4.59671021e-03,  2.11181641e-02,  6.30569458e-03],
        [-9.43756104e-03, -5.28335571e-03,  6.21795654e-04, ...,
          5.61141968e-03,  3.46374512e-03,  5.43975830e-03],
        ...,
        [-1.63841248e-03, -8.97216797e-03,  6.09207153e-03, ...,
          9.46807861e-03, -5.98907471e-04, -7.24792480e-04],
        [-4.78363037e-03, -6.96182251e-03, -3.38172913e-03, ...,
          1.01623535e-02, -6.74247742e-04,  3.77464294e-03],
        [-2.94923782e-04, -2.40898132e-03,  1.34277344e-02, ...,
          8.56018066e-03,  7.11059570e-03,  1.46865845e-03]]],
      dtype=float32)
Coordinates:
  * trials   (trials) int64 2kB 0 1 2 3 4 5 6 7 ... 217 218 219 220 221 222 223
  * time     (time) float64 160B -10.0 -8.947 -7.895 -6.842 ... 7.895 8.947 10.0
  * units    (units) int64 40kB 0 1 2 3 4 5 6 ... 4994 4995 4996 4997 4998 4999

In [15]:
white_noise_data

<xarray.DataArray (time: 20, units: 126467)> Size: 20MB
array([[ 0.6422616 , -0.98710396, -0.48565405, ..., -0.94346787,
        -1.1541933 , -1.24857038],
       [-0.60692105, -0.960001  , -0.08745389, ..., -1.5761281 ,
        -1.35232614, -0.32911346],
       [-1.16641362,  0.34336301,  1.27040938, ...,  1.280483  ,
        -0.42905716, -0.66857579],
       ...,
       [ 0.76860002, -0.53109292,  2.12925377, ..., -0.12825393,
         0.20633297, -0.59547157],
       [-1.31672874,  0.35222122,  1.93810119, ...,  0.05467622,
        -0.57558303, -0.62588208],
       [ 1.07181058,  0.03013764, -0.69615161, ..., -1.4688641 ,
         0.68499127, -0.78341647]])
Coordinates:
  * time     (time) float64 160B -10.0 -8.947 -7.895 -6.842 ... 7.895 8.947 10.0
  * units    (units) int64 1MB 0 1 2 3 4 ... 126462 126463 126464 126465 126466

In [16]:
map_kwargs = {"map_type": "percentile"}
# turn it into immutable tuple to store as a key
map_kwargs_key = convert_dict_to_tuple(map_kwargs)


source_resp = white_noise_data[:, 0:5000]
target_resp = target_brain_data[:,:,0:5000]

results = get_linregress_consistency(
    source=source_resp,
    target=target_resp,
    map_kwargs=map_kwargs,
    num_bootstrap_iters=1,
    num_parallel_jobs=1,
    splits=None,
    train_frac=0.5,
    num_train_test_splits=5,
)


/home/rdkeller/miniconda3/envs/fishies/lib/python3.12/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in scalar power
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/home/rdkeller/miniconda3/envs/fishies/lib/python3.12/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in scalar power
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/home/rdkeller/miniconda3/envs/fishies/lib/python3.12/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in scalar power
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/home/rdkeller/miniconda3/envs/fishies/lib/python3.12/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in scalar power
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/home/rdkeller/miniconda3/envs/fishies/lib/python3.12/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in sca

In [1]:
np.nanmedian(results['test']['r_xy_n_sb'])

NameError: name 'np' is not defined